In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
import os
import gc # 메모리 챙길 때 씀
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings

In [29]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [30]:
pd.set_option('display.max_rows', 60)

In [31]:
pd.set_option('display.max_columns', 100)

데이터가 너무 커서 읽는데 시간이 너무 오래 걸립니다. 따라서 30000개만 읽어 디버깅을 해줘야 합니다.

In [32]:
debug = True

In [33]:
num_rows = 30000 if debug else None

df = get_train_test(DATA_DIRECTORY, num_rows = num_rows) # 함수 뜯어보기

In [34]:
# GENERAL CONFIGURATIONS
NUM_THRADS = 4
DATA_DIRECTORY = "../input/"
SUBMISSION_SUFIX = "_model2_04"

In [35]:
path = DATA_DIRECTORY
num_rows = num_rows

In [36]:
train_file_path = '../input/home-credit-default-risk/application_train.csv'
test_file_path = '../input/home-credit-default-risk/application_test.csv'

In [37]:
train = pd.read_csv(train_file_path, nrows = num_rows)
test = pd.read_csv(test_file_path, nrows = num_rows)

In [38]:
df = train.append(test)

In [39]:
train

In [40]:
df

In [41]:
del train, test

메모리를 주워담습니다.

In [42]:
gc.collect

## preprocessing

In [43]:
df = df[df['CODE_GENDER'] != 'XNA']# XNA에는 4명의 사람 밖에 없기 때문에 제거합니다.

In [44]:
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # test의 Max income은 4백만, train은 1억7백만입니다.

In [45]:
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # 365243으로 기록된 사람들이 많습니다. 이는 의미 없는 숫자이므로 nan값을 적용하겠습니다.

In [46]:
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [47]:
docs = [f for f in df.columns if 'FLAG_DOC' in f] # 이들은 binary data입니다. "에, 아니오" 등으로 이루어진 문서

In [48]:
df['DOCUMENT_COUNT'] = df[docs].sum(axis=1)
df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1)

In [49]:
df[docs].sum(axis=1)

In [50]:
df[docs].kurtosis(axis=1).hist() # kurtosis는 분포의 꼬리를 의미합니다.

In [51]:
# df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))
# get_age_label 함수의 df['DAYS_BIRTH']를 보면 -살아온 일 수가 기록돼 있음.
# 여기다 -를 취해 양수로 만들어준 이후 365로 나누면 나이가 나옴. 이를 구간으로 나눠준 함수

In [52]:
# EXT_SOURCE는 feature importance가 높게 나오는 값들입니다. 이들끼리 서로 interaction할 수 있게 곱해서 새로운 feature를 만들어줍니다.
df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

# 현업에서는 설명이 되지 않는 변수는 사용하기 어렵지만 캐글에서는 점수를 높여야 하니 사용하는 경향이 있습니다.

In [57]:
# 경진대회에 참여하는 사람들이 해당 feature들 간의 상관관계를 조사해본 결과 2, 1, 3이 좋게 나타났습니다.
df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3

In [58]:
np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')

In [59]:
# eval 함수는 string 형태를 evaluation할 수 있도록 만들어줌

for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
    feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
    df[feature_name] = eval('np.{}'.format(function_name))(
        df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

In [60]:
# Credit ratios
df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY'] # 연금을 받는데 빛은 얼마나 지는지
df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE'] # 물건을 사는데 빚을 얼마나 지는지

In [61]:
# Income ratios
df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED'] # 일한 햇수에 비해 임금이 얼만지
df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']

In [62]:
# Time ratios
df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH'] # 살아온 날 대비 얼마나 근로했는지
df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH'] # 회사에 아이디 만들어진지 얼마나 되는지
df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH'] # 나이 대비 car를 소유한 일 수
df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']